In [7]:
import pandas as pd
import datetime as pytz
from uszipcode import SearchEngine                         # !pip3 install uszipcode
search = SearchEngine(simple_zipcode=False)                # set simple_zipcode=False to use rich info database
# !pip3 install --upgrade uszipcode                          # upgrade databases
from simple_salesforce import Salesforce                   # !pip3 install simple-salesforce
from salesforce_reporting import Connection, ReportParser  # !pip3 install salesforce_reporting
with open('.sfdc') as f:
    uname, spass, stoken, ppass, ptoken = [x.strip("\n") for x in f.readlines()]

In [8]:
sf = Salesforce(username=uname, password=spass, security_token=stoken)          # authenticate
end = datetime.datetime.now(pytz.UTC)                                           # salesforce API requires UTC
# print("Session ID:", session_id)
# print("Instance:", sf_instance)
# print("Version:", sf_version)
print("@ ", end)

SalesforceAuthenticationFailed: API_DISABLED_FOR_ORG: API is not enabled for this Organization or Partner

In [181]:
acctdf = sf.bulk.Account.query("SELECT Name, Id, OwnerID, BDRAssigned__c FROM Account WHERE CAPDB__c <> '' AND Closed_Won_Opportunity_Count__c = 0 AND of_Active_Opps__c =  0 AND of_Opportunities__c > 0 AND BDRAssigned__c <> 'None Assigned' LIMIT 10")
records = [dict(
        Id = rec['Id'],
   OwnerID = rec['BDRAssigned__c'])
    for rec in query['records']]
acctdf = pd.DataFrame(records)
acctdf.to_csv (r'CAPDB_CL.csv', index = None, header=True)

# acctdf_j = acctdf.to_json(orient='records')
# acctdf_j
# sf.bulk.Account.update(acctdf_j)           # sf.Contact.update('003e0000003GuNXAA0',{'LastName': 'Jones', 'FirstName': 'John'})

In [125]:
query = sf.query("SELECT Id, PostalCode, StateCode FROM Lead WHERE LeadSource = 'Madison'")
records = [dict(
            Id = rec['Id'],
    PostalCode = rec['PostalCode'],
     StateCode = rec['StateCode'])
    for rec in query['records']]
qdf = pd.DataFrame(records)
qdf['PostalCode'] = qdf['PostalCode'].str[:5]       # trim off zip+4
def zco(x):
    return search.by_zipcode(x).state
qdf['StateCode'] = qdf['PostalCode'].fillna(0).astype(int).apply(zco)
qdf.to_csv (r'madison.csv', index = None, header=True
# qdfj = qdf.to_json(orient='records')
# qdfj
# qdf
# sf.bulk.Contact.insert(data)
# sf.bulk.Contact.upsert(data, 'Id')
# sf.bulk.Lead.update(qdfj)

### <a href="https://developer.salesforce.com/docs/api-explorer"><img src="img/l_sfdev.png" style="width:200px; float: left;"></a>    
<h2 style="padding-left:225px; border:none">Salesforce Methods</h2>
  
--------------------------------------  

Create file /root/.sfdc with credentials as:  
username  
password  
token  
uid
instance

### authenticate 
```python
from salesforce_reporting import Connection, ReportParser
with open('.sfdc') as f:
    username, password, token, uid, instance = [x.strip("\n") for x in f.readlines()]
sf = Salesforce(username=username, password=password, security_token=token, sandbox=False)
```

### query
```python
ContactDict = sf.query("SELECT Company, CreatedDate, Email, FirstName, Industry, LastName, LeadSource, Title FROM Lead ORDER BY CreatedDate DESC NULLS FIRST LIMIT 50")

opportunity_amounts = sf.query("SELECT Id, Probability, StageName, Amount, Owner.Name FROM Opportunity WHERE AMOUNT < 10000")

amounts = [x['Amount'] for x in opportunity_amounts['records']]
owners = [x['Owner']['Name'] for x in opportunity_amounts['records']]
hist1 = go.Histogram(x=amounts)
py.iplot([hist1], filename='salesforce/opportunity-probability-histogram')
```

sf.Contact.updated(end - datetime.timedelta(days=10), end)       # contacts updated last 10d
contact = sf.Contact.get('003e0000003GuNXAA0')                   # 18 digit id
metadata = sf.Contact.metadata()
describe = sf.Contact.describe()

### read into dataframe
```python
df = pd.DataFrame(ContactDict['records'])
df.head(10)
Age = df['Age__c']
Age.head(10)
agePlot = Age.hist(bins=10)
```

### process
```python
lead_comparison = []
for name, vals in df.groupby('Owners'):
    counts = vals.Status.value_counts()
    lead_comparison.append(Bar(x=counts.index, y=counts.values, name=name))

scatters = []
for name, temp_df in large_opps_df.groupby('Owner'):
    hover_text = temp_df.Name + "<br>Close Probability: " + temp_df.Probability.map(str) + "<br>Stage:" + temp_df.StageName
    scatters.append(
        go.Scatter(
            x=temp_df.CloseDate,
            y=temp_df.Amount,
            mode='markers',
            name=name,
            text=hover_text,
            marker=dict(
                size=(temp_df.Probability / 2) # helps keep the bubbles of managable size
            )
        )
    )

data = scatters
layout = go.Layout(
    title='Open Large Deals',
    xaxis=dict(
        title='Close Date'
    ),
    yaxis=dict(
        title='Deal Amount',
        showgrid=False
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='salesforce/open-large-deals-scatter')
```

### reports

```python
!pip install salesforce-analytics
```

[salesforce-reporting](https://github.com/cghall/force-retrieve)

```python
!pip install salesforce-reporting

report = sf.get_report('report_id', includeDetails=True)
parser = salesforce_reporting.ReportParser(report)
parser.records()
```
  
```python
report = my_sf.get_report('report_id')
matrix_parser = salesforce_reporting.MatrixParser(report)
matrix_parser.series_down('Jan 2016')
```

### Salesforce Streaming Client
A Salesforce streaming API client for python, built on salesforce-requests-oauthlib and python-bayeux.

To run tests, install py.test and pytest-cov in your virtualenv

py.test --cov=src/salesforce_streaming_client/ --cov-report html:coverage  

View test coverage results ./coverage

```python  
pip3 install salesforce-streaming-client
```  

### [Pardot](https://github.com/mneedham91/PyPardot4)

Version 4 for duped email Pardot configurations

**Install**  
_Requires [requests](http://docs.python-requests.org/en/latest/)_

```python
$ !pip install pypardot4
```
  
  
|Parameter|	Required|Description|  
|---------|---------|-----------|  
|object|X|The object type to be returned by the API request|  
|op|X|The operation to be performed on the specified object type|  
|id_field|X|The field to be used as the identifier for the specified object|  
|id|X|The identifier for the specified object(s)|  
|your_api_key|X|The API key that was obtained during Authentication|  
|your_user_key|X|The user key that was used during Authentication|  
|format|The API data format. Either xml or json (xml is default)|  
|params|Parameters specific to your request; See individual methods for details|  

**Authenticate**
```python
pa = PardotAPI(email='email',password='password',user_key='user_key')  
pa.authenticate()
```

**Create a new prospect**
```python
pa.prospects.create(email='joe@company.com', first_name='Joe', last_name='Schmoe')
```

**Read data about our prospect**
```python
print(pa.prospects.read_by_email(email='joe@company.com'))
```

**Query and iterate through today's prospects**
Pardot only returns 200 records with each request  
Use offset to retrieve matching records beyond this limit  

```python
prospects = pa.prospects.query(created_after='yesterday')
total = prospects['total_results'] # total number of matching records
for prospect in prospects['prospect']
  print(prospect.get('first_name'))
```
  
**Create a new prospect**
```python
pa.prospects.create_by_email(email='joe@company.com', first_name='Joe', last_name='Schmoe')
```

**Update a prospect field (works with default or custom field)**
```python
pa.prospects.update_field_by_id(id=23839663, field='company', field_value='Joes Plumbing')
```

**Send a one-off email**
```python
pa.emails.send_to_email(prospect_email='joe@company.com', email_template_id=123)
```

**Performing API calls is inherently unsafe, so be sure to catch exceptions**
```python
try:
  pa.prospects.create_by_email(email='existing.email.address@company.com')
except PardotAPIError, e:
  print(e)
```


In [ ]:
search = SearchEngine(simple_zipcode=True)                # False uses rich info database
zipcode = search.by_zipcode("06916")
# zipcode.to_json()                                       # to json
zipcode.to_dict()                                         # to dict
# zip = zipcode.values()                                  # to list
zipcode
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}